In [ ]:
# TODO Não precisa fazer resample, pois dataset já é 16kHZ!

In [27]:
import pandas as pd
import glob
import torchaudio
import re
SAMPLE_RATE=16000

In [2]:
# https://github.com/rmarcacini/ser-coraa-pt-br/ to urbansounds


Meta-data Files Included
------------------------

UrbanSound8k.csv

This file contains meta-data information about every audio file in the dataset. This includes:  

* slice_file_name:  
The name of the audio file. The name takes the following format: [fsID]-[classID]-[occurrenceID]-[sliceID].wav, where:  
[fsID] = the Freesound ID of the recording from which this excerpt (slice) is taken      
[classID] = a numeric identifier of the sound class (see description of classID below for further details)  
[occurrenceID] = a numeric identifier to distinguish different occurrences of the sound within the original recording  
[sliceID] = a numeric identifier to distinguish different slices taken from the same occurrence  

* fsID:
The Freesound ID of the recording from which this excerpt (slice) is taken  

* start
The start time of the slice in the original Freesound recording  

* end:
The end time of slice in the original Freesound recording  

* salience:
A (subjective) salience rating of the sound. 1 = foreground, 2 = background.  

* fold:
The fold number (1-10) to which this file has been allocated.  

* classID:  
A numeric identifier of the sound class:  
0 = air_conditioner  
1 = car_horn  
2 = children_playing  
3 = dog_bark  
4 = drilling  
5 = engine_idling  
6 = gun_shot  
7 = jackhammer  
8 = siren  
9 = street_music  

* class:
The class name: air_conditioner, car_horn, children_playing, dog_bark, drilling, engine_idling, gun_shot, jackhammer,
siren, street_music.  



In [3]:
# Todo criar Pasta no formato urban sound
# Criar CSv com metadados

In [28]:
audios = glob.glob('../../data/train/*.wav')

In [29]:
df = pd.DataFrame()

In [30]:
str_exemple = audios[0]
print(str_exemple)
pattern = ".*train/(?P<sess_id>.*)_(?P<utt_id>.*)_(?P<class>.*).wav"
print('pattern:',pattern)

../../data/train/bfamcv01_segment163_neutral.wav
pattern: .*train/(?P<sess_id>.*)_(?P<utt_id>.*)_(?P<class>.*).wav


In [31]:
df = pd.DataFrame({'audio' : audios})
df = pd.concat([df,df.audio.str.extract(pattern)], axis = 1)
df.head(1)

,audio,sess_id,utt_id,class
0,../../data/train/bfamcv01_segment163_neutral.wav,bfamcv01,segment163,neutral


In [33]:
map_sess_id = {j: i for i,j in enumerate(df.sess_id.unique())}
map_class = {j: i for i,j in enumerate(df['class'].unique())}

In [34]:
print(map_class)

{'neutral': 0, 'non-neutral-male': 1, 'non-neutral-female': 2}


In [35]:
df['fsID'] = df['sess_id'].map(map_sess_id)
df['classID'] = df['class'].map(map_class)
df['occurrenceID'] = df['utt_id'].apply(lambda x: x.split('segment')[1])
df['sliceID'] = 0
df['salience'] = 1

In [39]:
df['slice_file_name'] = df.apply(lambda row:  f"{row['fsID']}-{row['classID']}-{row['occurrenceID']}-{row['sliceID']}.wav"
                                 , axis = 1)

In [41]:
df['start'] = 0
df['stop'] = df.audio.apply(lambda x: torchaudio.load(x)[0].shape[1])
df['end'] = df.stop.apply(lambda x: x/SAMPLE_RATE).astype(str)

In [117]:
import numpy as np
from sklearn.model_selection import StratifiedKFold 
X = df
y = df.classID

sss = StratifiedKFold(n_splits=10, random_state=0)
sss.get_n_splits(X, y)

df['fold'] = 'NA'
i = 1
for train_index, test_index in sss.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    df.loc[test_index, 'fold'] = i
    print(df.loc[test_index]['class'].value_counts())
    i=i+1

neutral               50
non-neutral-female     9
non-neutral-male       4
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       5
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       5
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       5
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       5
Name: class, dtype: int64
neutral               49
non-neutral-female     8
non-neutral-male       5
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       4
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       4
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       4
Name: class, dtype: int64
neutral               49
non-neutral-female     9
non-neutral-male       4
Name: class, dty

/home/victor/anaconda3/envs/torchaudio/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


In [118]:
df.fold.value_counts()

5     63
4     63
3     63
2     63
1     63
10    62
9     62
8     62
7     62
6     62
Name: fold, dtype: int64

In [ ]:
# Fodo function that copy files to urband sound format and the metadata csv

In [104]:
df_metadata = df[['slice_file_name','fsID','start','end','salience','fold','classID','class']]